In [ ]:
import concurrent.futures
import os

import dotenv
import polars as pl

from libraries.client_stashapp import StashAppClient, get_stashapp_client
from libraries.StashDbClient import StashDbClient


def get_stashdb_data(stashdb_id):
    return stashbox_client.query_scenes_by_performer(stashdb_id)

def parallel_fetch_stashdb_data(stashdb_ids):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        return list(executor.map(get_stashdb_data, stashdb_ids))


dotenv.load_dotenv()

stash = get_stashapp_client()
stash_client = StashAppClient()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [ ]:
def get_scenes_with_stashdb_id():
    scenes = stash.find_scenes(
        {
            "stash_id_endpoint": {
                "endpoint": "https://stashdb.org/graphql",
                "modifier": "NOT_NULL",
            }
        },
        fragment=(
            "id title details"
            " studio { id name stash_ids { stash_id endpoint } }"
            " performers { id name stash_ids { stash_id endpoint } }"
            " stash_ids { stash_id endpoint }"
            " tags { id name aliases }"
            " files { fingerprints { type value } }"
        ),
    )
    return scenes

def get_primary_file_phash(files):
    primary_file = files[0]
    phash = None
    for fingerprint in primary_file["fingerprints"]:
        if fingerprint["type"] == "phash":
            phash = fingerprint["value"]
            break
    return phash

scenes_with_stashdb_id = get_scenes_with_stashdb_id()

for scene in scenes_with_stashdb_id:
    # Get StashDB ID from tag aliases
    scene["tags"] = [
        {
            "id": tag["id"],
            "name": tag["name"],
            "stashdb_id": next(
                (
                    alias[len("StashDB ID: "):]
                    for alias in tag["aliases"]
                    if alias.startswith("StashDB ID: ")
                ),
                None,
            ),
        }
        for tag in scene["tags"]
    ]

df_scenes = pl.DataFrame(scenes_with_stashdb_id).with_columns(
    pl.col("stash_ids")
    .map_elements(
        lambda x: next(
            (
                stash_id["stash_id"]
                for stash_id in x
                if stash_id["endpoint"] == "https://stashdb.org/graphql"
            ),
            None,
        ),
        return_dtype=pl.Utf8,
    )
    .alias("stashdb_id"),

    pl.col("tags")
    .map_elements(
        lambda x: sorted([tag["name"] for tag in x]), return_dtype=pl.List(pl.Utf8)
    )
    .alias("existing_tags"),

    pl.col("performers")
    .map_elements(
        lambda x: list(
            filter(
                None,
                [
                    next(
                        (
                            stash_id["stash_id"]
                            for stash_id in performer.get("stash_ids", [])
                            if stash_id["endpoint"] == "https://stashdb.org/graphql"
                        ),
                        None,
                    )
                    for performer in x
                ],
            )
        ),
        return_dtype=pl.List(pl.Utf8),
    )
    .alias("existing_performers"),

    pl.col("studio").map_elements(
        lambda x: x["id"] if x else None, return_dtype=pl.Utf8
    ).alias("studio_id"),
    pl.col("studio").map_elements(
        lambda x: x["name"] if x else None, return_dtype=pl.Utf8
    ).alias("studio_name"),
    pl.col("studio").map_elements(
        lambda x: next(
            (
                stash_id["stash_id"]
                for stash_id in x["stash_ids"]
                if stash_id["endpoint"] == "https://stashdb.org/graphql"
            ),
            None,
        ) if x else None,
        return_dtype=pl.Utf8,
    ).alias("studio_stashdb_id"),
    pl.col("files").map_elements(
        lambda x: get_primary_file_phash(x), return_dtype=pl.Utf8
    ).alias("phash")
).drop("studio")
df_scenes

In [41]:
# Step 3: Fetch StashDB data for each scene
df_performers = df_scenes.select("id", "performers").explode("performers").with_columns([
    pl.col("performers").struct.field("id").alias("performer_id"),
    pl.col("performers").struct.field("name").alias("performer_name"),
    pl.col("performers").struct.field("stash_ids").list.eval(
        pl.element().struct.field("stash_id").filter(
            pl.element().struct.field("endpoint") == "https://stashdb.org/graphql"
        )
    ).list.first().alias("stashdb_id")
]).select("performer_id", "performer_name", "stashdb_id").unique().head(10)

In [ ]:
unique_scene_stashdb_ids_and_phashes = df_scenes.select("stashdb_id", "phash").unique().head(100).to_dicts()
unique_scene_stashdb_ids_and_phashes

In [ ]:
unique_scene_ids = df_scenes.select("stashdb_id").unique().to_series().drop_nulls().to_list()
df_stashdb_results = stashbox_client.query_scenes_by_phash_optimized(unique_scene_stashdb_ids_and_phashes)
df_stashdb_results = df_stashdb_results.with_columns([
    pl.col("performers").list.eval(
        pl.element().struct.field("performer")
    ).alias("performers")
])
df_stashdb_results

In [44]:
df_mapped_stashdb_results = df_stashdb_results.with_columns([
    # Extract both id and name as a struct from tags
    pl.col("tags").map_elements(
        lambda x: [{"id": tag["id"], "name": tag["name"]} for tag in x],
        return_dtype=pl.List(pl.Struct([
            pl.Field("id", pl.Utf8),
            pl.Field("name", pl.Utf8)
        ]))
    ).alias("stashdb_tags"),

    # Extract performer data similarly
    pl.col("performers").map_elements(
        lambda x: [{"id": p["id"], "name": p["name"]} for p in x],
        return_dtype=pl.List(pl.Struct([
            pl.Field("id", pl.Utf8),
            pl.Field("name", pl.Utf8)
        ]))
    ).alias("stashdb_performers")
])

In [51]:
df_tags = stash_client.get_tags()

In [53]:
df_unique_tags_in_stashdb_scenes = df_mapped_stashdb_results.select("stashdb_tags").explode("stashdb_tags").select(
    pl.col("stashdb_tags").struct.field("id").alias("stashdb_tag_id"),
    pl.col("stashdb_tags").struct.field("name").alias("stashdb_tag_name"),
).unique().drop_nulls()

In [ ]:
df_joined_tags = df_unique_tags_in_stashdb_scenes.join(df_tags, left_on="stashdb_tag_id", right_on="stashdb_id", how="left")
df_joined_tags

In [55]:
df_performers = stash_client.get_performers()

In [56]:
df_unique_performers_in_stashdb_scenes = df_mapped_stashdb_results.select("stashdb_performers").explode("stashdb_performers").select(
    pl.col("stashdb_performers").struct.field("id").alias("stashdb_performer_id"),
    pl.col("stashdb_performers").struct.field("name").alias("stashdb_performer_name")
).unique().drop_nulls()

In [ ]:
df_joined_performers = df_unique_performers_in_stashdb_scenes.join(
    df_performers, left_on="stashdb_performer_id",
    right_on="stashapp_stashdb_id", how="left",
)
df_joined_performers.filter(pl.col("stashapp_id").is_null())

In [ ]:
df_foobar = df_scenes.join(df_mapped_stashdb_results, left_on="stashdb_id", right_on="id", how="left")
df_foobar = df_foobar.filter(pl.col("title_right").is_not_null())
df_foobar

In [30]:
# Create comparison columns for performers and tags
df_foobar2 = df_foobar.with_columns([
    # Compare performers - extract just the IDs from stashdb_performers structs
    pl.struct(["existing_performers", "stashdb_performers"]).map_elements(
        lambda x: list(set([p["id"] for p in x["stashdb_performers"]]) - set(x["existing_performers"]))
    , return_dtype=pl.List(pl.Utf8)).alias("incoming_performers"),

    pl.struct(["existing_performers", "stashdb_performers"]).map_elements(
        lambda x: list(set(x["existing_performers"]) - set([p["id"] for p in x["stashdb_performers"]]))
    , return_dtype=pl.List(pl.Utf8)).alias("outgoing_performers"),

    # Compare tags - extract just the names from stashdb_tags structs
    pl.struct(["existing_tags", "stashdb_tags"]).map_elements(
        lambda x: list(set([t["name"] for t in x["stashdb_tags"]]) - set(x["existing_tags"]))
    , return_dtype=pl.List(pl.Utf8)).alias("incoming_tags"),

    pl.struct(["existing_tags", "stashdb_tags"]).map_elements(
        lambda x: list(set(x["existing_tags"]) - set([t["name"] for t in x["stashdb_tags"]]))
    , return_dtype=pl.List(pl.Utf8)).alias("outgoing_tags")
])

# Add boolean flags for quick filtering
df_foobar2 = df_foobar2.with_columns([
    pl.col("incoming_performers").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_incoming_performers"),

    pl.col("outgoing_performers").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_outgoing_performers"),

    pl.col("incoming_tags").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_incoming_tags"),

    pl.col("outgoing_tags").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_outgoing_tags")
])

In [ ]:
df_foobar2 = df_foobar2.filter(
    pl.col("has_incoming_performers")
    | pl.col("has_outgoing_performers")
    | pl.col("has_incoming_tags")
    | pl.col("has_outgoing_tags")
)
df_foobar2

In [ ]:
df_stash_performers = pl.DataFrame(stash.find_performers(
    {
        "stash_id_endpoint": {
            "endpoint": "https://stashdb.org/graphql",
            "modifier": "NOT_NULL",
            "stash_id": "",
        }
    },
    fragment="id name stash_ids { stash_id endpoint }",
))
df_stash_performers = df_stash_performers.with_columns(
    pl.col("stash_ids").list.eval(
        pl.element().struct.field("stash_id").filter(
            pl.element().struct.field("endpoint") == "https://stashdb.org/graphql"
        )
    ).list.first().alias("stashdb_id")
).drop("stash_ids")
df_stash_performers

In [ ]:
df_scenes.schema

In [ ]:
# Map StashDB data to new columns
def extract_stashdb_info(stashdb_data):
    if stashdb_data and "data" in stashdb_data and "findScene" in stashdb_data["data"]:
        scene = stashdb_data["data"]["findScene"]
        return {
            "stashdb_title": scene.get("title", ""),
            "stashdb_details": scene.get("details", ""),
            "stashdb_release_date": scene.get("release_date", None),
            "stashdb_duration": scene.get("duration", None),
            "stashdb_code": scene.get("code", ""),
            "stashdb_performer_ids": [p["performer"]["id"] for p in scene.get("performers", [])],
            "stashdb_performer_names": [p["performer"]["name"] for p in scene.get("performers", [])],
            "stashdb_tag_ids": [t["id"] for t in scene.get("tags", [])],
            "stashdb_tag_names": [t["name"] for t in scene.get("tags", [])]
        }
    return {
        "stashdb_title": None,
        "stashdb_details": None,
        "stashdb_release_date": None,
        "stashdb_duration": None,
        "stashdb_code": None,
        "stashdb_performer_ids": [],
        "stashdb_performer_names": [],
        "stashdb_tag_ids": [],
        "stashdb_tag_names": []
    }

# Add mapped columns to dataframe
df_mapped_scenes = df_scenes.with_columns([
    pl.col("stashdb_data").map_elements(
        lambda x: extract_stashdb_info(x),
        return_dtype=pl.Struct({
            "stashdb_title": pl.Utf8,
            "stashdb_details": pl.Utf8,
            "stashdb_release_date": pl.Date,
            "stashdb_duration": pl.Int32,
            "stashdb_code": pl.Utf8,
            "stashdb_performer_ids": pl.List(pl.Utf8),
            "stashdb_performer_names": pl.List(pl.Utf8),
            "stashdb_tag_ids": pl.List(pl.Utf8),
            "stashdb_tag_names": pl.List(pl.Utf8)
        })
    ).alias("stashdb_info")
]).unnest("stashdb_info")

# Add comparison columns
df_mapped_scenes = df_mapped_scenes.with_columns([
    # Compare existing vs stashdb performers
    pl.struct(["existing_performers", "stashdb_performer_ids"]).map_elements(
        lambda x: list(set(x["stashdb_performer_ids"]) - set(x["existing_performers"]))
    , return_dtype=pl.List(pl.Utf8)).alias("incoming_performers"),

    pl.struct(["existing_performers", "stashdb_performer_ids"]).map_elements(
        lambda x: list(set(x["existing_performers"]) - set(x["stashdb_performer_ids"]))
    , return_dtype=pl.List(pl.Utf8)).alias("outgoing_performers"),

    # Compare existing vs stashdb tags
    pl.struct(["existing_tags", "stashdb_tag_names"]).map_elements(
        lambda x: list(set(x["stashdb_tag_names"]) - set(x["existing_tags"]))
    , return_dtype=pl.List(pl.Utf8)).alias("incoming_tags"),

    pl.struct(["existing_tags", "stashdb_tag_names"]).map_elements(
        lambda x: list(set(x["existing_tags"]) - set(x["stashdb_tag_names"]))
    , return_dtype=pl.List(pl.Utf8)).alias("outgoing_tags"),
])

# Add boolean flags
df_mapped_scenes = df_mapped_scenes.with_columns([
    pl.col("incoming_performers").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_incoming_performers"),

    pl.col("outgoing_performers").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_outgoing_performers"),

    pl.col("incoming_tags").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_incoming_tags"),

    pl.col("outgoing_tags").map_elements(
        lambda x: len(x) > 0,
        return_dtype=pl.Boolean
    ).alias("has_outgoing_tags")
])

In [ ]:


# Get unique stashdb_ids and fetch their data
stashdb_ids = df_performers.select("stashdb_id").unique().to_series().drop_nulls().to_list()
# Get ID from data
stashdb_data_map = {data["id"]: data for data in stashdb_results}

# Add the data back to the dataframe using a mapping function
df_scenes = df_scenes.head(10).with_columns([
    pl.col("incoming_performers").map_elements(
        lambda x: [stashdb_data_map.get(stashdb_id) for stashdb_id in x],
        return_dtype=pl.List(pl.Struct)
    ).alias("stashdb_data")
])

In [ ]:
emily_pink_results = get_stashdb_data("42b23510-5a9f-4f66-8d83-5bdb56454ff1")

In [ ]:
pl.DataFrame(emily_pink_results).head(1)


In [ ]:
# Step 3: Fetch StashDB data for each scene


def get_stashdb_data(stashdb_id):
    return stashbox_client.query_scenes_by_performer(stashdb_id)

def parallel_fetch_stashdb_data(stashdb_ids):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        return list(executor.map(get_stashdb_data, stashdb_ids))

stashdb_ids = df_scenes.select("stashdb_id").unique().to_series().to_list()
df_scenes["stashdb_data"] = parallel_fetch_stashdb_data(stashdb_ids)

In [ ]:
import json
import os
from pathlib import Path

import requests


# Define the directory to store JSON files and images
json_dir = r"X:\StashDB JSON"

# Create the directory if it doesn't exist
Path(json_dir).mkdir(parents=True, exist_ok=True)

# Function to save JSON data
def save_json(stashdb_id, data):
    filename = f"{stashdb_id}.json"
    filepath = Path(json_dir) / filename
    with filepath.open("w") as f:
        json.dump(data, f, indent=2)

# Function to download and save the first image
def save_first_image(stashdb_id, image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        content_type = response.headers.get("content-type")
        if content_type:
            ext = content_type.split("/")[-1]
            filename = f"{stashdb_id}.{ext}"
            filepath = Path(json_dir) / filename
            with filepath.open("wb") as f:
                f.write(response.content)
            print(f"Saved image: {filename}")
        else:
            print(f"Could not determine file type for image: {image_url}")
    else:
        print(f"Failed to download image: {image_url}")

# Save StashDB data and first image for each scene
for stashdb_id, stashdb_data in zip(df_scenes["stashdb_id"], df_scenes["stashdb_data"], strict=False):
    if stashdb_id and stashdb_data:
        save_json(stashdb_id, stashdb_data)

        # Check if there are images and download the first one
        if "data" in stashdb_data and "findScene" in stashdb_data["data"]:
            images = stashdb_data["data"]["findScene"].get("images", [])
            if images:
                first_image_url = images[0].get("url")
                if first_image_url:
                    save_first_image(stashdb_id, first_image_url)

print(f"Saved StashDB JSON responses and first images to {json_dir}")

In [ ]:
# Step 4: Extract performers from StashDB data
def extract_performers(stashdb_data):
    if stashdb_data and "data" in stashdb_data and "findScene" in stashdb_data["data"]:
        return list(set([performer["performer"]["id"] for performer in stashdb_data["data"]["findScene"]["performers"]]))
    return []

df_scenes = df_scenes.with_columns([
    pl.col("stashdb_data").map_elements(extract_performers, return_dtype=pl.List(pl.Utf8)).alias("stashdb_performers")
])

# Step 5: Create a dataframe for performers
df_performers = df_scenes.select([
    "id",
    "existing_performers",
    "stashdb_performers"
])

# Step 6: Identify missing and extra performers
df_performers = df_performers.with_columns([
    pl.struct(["stashdb_performers", "existing_performers"]).map_elements(
        lambda x: list(set(x["stashdb_performers"]) - set(x["existing_performers"]))
    , return_dtype=pl.List(pl.Utf8)).alias("missing_performers"),
    pl.struct(["existing_performers", "stashdb_performers"]).map_elements(
        lambda x: list(set(x["existing_performers"]) - set(x["stashdb_performers"]))
    , return_dtype=pl.List(pl.Utf8)).alias("extra_performers")
])

# Step 7: Find corresponding performers in local Stash
def find_stash_performers(performer_ids):
    return [
        stash.find_performers({
            "stash_id_endpoint": {
                "endpoint": "https://stashdb.org/graphql",
                "stash_id": performer_id,
                "modifier": "EQUALS"
            }
        })
        for performer_id in performer_ids
        if performer_id
    ]

df_performers = df_performers.with_columns([
    pl.col("stashdb_performers").map_elements(find_stash_performers, return_dtype=pl.List(pl.Struct)).alias("stash_performers")
])

# Step 8: Add boolean columns for performers
df_performers = df_performers.with_columns([
    pl.col("extra_performers").map_elements(lambda x: len(x) > 0, return_dtype=pl.Boolean).alias("has_local_performers_not_in_stashdb"),
    pl.col("missing_performers").map_elements(lambda x: len(x) > 0, return_dtype=pl.Boolean).alias("has_stashdb_performers_not_in_local")
])

In [15]:
# Create a backup copy of the original df_scenes
# df_scenes_original = df_scenes.copy()

In [ ]:


# Create a new df_scenes with only 100 scenes
df_scenes = df_scenes_original.copy()

print(f"Original number of scenes: {len(df_scenes_original)}")
print(f"New number of scenes: {len(df_scenes)}")


In [ ]:
# Step 4: Extract performers from StashDB data
def extract_performers(stashdb_data):
    if stashdb_data and "data" in stashdb_data and "findScene" in stashdb_data["data"]:
        return set([performer["performer"]["id"] for performer in stashdb_data["data"]["findScene"]["performers"]])
    return []

df_scenes["stashdb_performers"] = df_scenes["stashdb_data"].apply(extract_performers)

# Step 5: Create a dataframe for performers
df_performers = pd.DataFrame({
    "scene_id": df_scenes["id"],
    "existing_performers": df_scenes["existing_performers"],
    "stashdb_performers": df_scenes["stashdb_performers"]
})

# Step 6: Identify missing and extra performers
df_performers["missing_performers"] = df_performers.apply(
    lambda row: set(row["stashdb_performers"]) - set(row["existing_performers"]),
    axis=1,
)
df_performers["extra_performers"] = df_performers.apply(
    lambda row: set(row["existing_performers"]) - set(row["stashdb_performers"]),
    axis=1,
)

# Step 7: Find corresponding performers in local Stash
def find_stash_performers(performer_ids):
    return [
        stash.find_performers({
            "stash_id_endpoint": {
                "endpoint": "https://stashdb.org/graphql",
                "stash_id": performer_id,
                "modifier": "EQUALS",
            }
        })
        for performer_id in performer_ids
        if performer_id
    ]

df_performers["stash_performers"] = df_performers["stashdb_performers"].apply(find_stash_performers)

# Step 8: Add boolean columns for performers
df_performers["has_local_performers_not_in_stashdb"] = df_performers["extra_performers"].apply(lambda x: len(x) > 0)
df_performers["has_stashdb_performers_not_in_local"] = df_performers["missing_performers"].apply(lambda x: len(x) > 0)

# Step 9: Select relevant columns for final dataframes
df_performers_selected = df_performers[[
    "scene_id", "existing_performers", "stashdb_performers",
    "missing_performers", "extra_performers", "stash_performers",
    "has_local_performers_not_in_stashdb",
    "has_stashdb_performers_not_in_local",
]]

In [29]:
tag_incoming_performers = stash.find_tag("StashDB: Incoming Performers")
tag_outgoing_performers = stash.find_tag("StashDB: Outgoing Performers")

In [ ]:
df_scenes_with_incoming_stashdb_performers = (
    df_performers_selected[df_performers_selected["has_stashdb_performers_not_in_local"]][["scene_id"]]
)

for scene_id in df_scenes_with_incoming_stashdb_performers["scene_id"]:
    scene = stash.find_scene(scene_id)
    current_tag_ids = [tag["id"] for tag in scene["tags"]]
    updated_tag_ids = [tag_incoming_performers["id"]] + current_tag_ids
    stash.update_scene({
        "id": scene_id,
        "tag_ids": updated_tag_ids
    })

In [ ]:
df_scenes_with_outgoing_stashdb_performers = (
    df_performers_selected[df_performers_selected["has_local_performers_not_in_stashdb"]][["scene_id"]]
)

for scene_id in df_scenes_with_outgoing_stashdb_performers["scene_id"]:
    scene = stash.find_scene(scene_id)
    current_tag_ids = [tag["id"] for tag in scene["tags"]]
    updated_tag_ids = [tag_outgoing_performers["id"]] + current_tag_ids
    stash.update_scene({
        "id": scene_id,
        "tag_ids": updated_tag_ids
    })

In [ ]:
# Step 4: Extract tags and performers from StashDB data
def extract_tags(stashdb_data):
    if stashdb_data and "data" in stashdb_data and "findScene" in stashdb_data["data"]:
        return sorted([tag["name"] for tag in stashdb_data["data"]["findScene"]["tags"]])
    return []

df_scenes["stashdb_tags"] = df_scenes["stashdb_data"].apply(extract_tags)

# Step 5: Create separate dataframes for tags and performers
df_tags = pd.DataFrame({
    "scene_id": df_scenes["id"],
    "existing_tags": df_scenes["existing_tags"],
    "stashdb_tags": df_scenes["stashdb_tags"]
})

# Step 6: Identify missing and extra tags/performers
df_tags["missing_tags"] = df_tags.apply(lambda row: sorted(set(row["stashdb_tags"]) - set(row["existing_tags"])), axis=1)
df_tags["extra_tags"] = df_tags.apply(lambda row: sorted(set(row["existing_tags"]) - set(row["stashdb_tags"])), axis=1)

# Step 7: Find corresponding tags and performers in local Stash
def find_stash_tags(tag_names):
    return sorted([stash.find_tag(tag_name) for tag_name in tag_names], key=lambda x: x["name"] if x else "")

df_tags["stash_tags"] = df_tags["stashdb_tags"].apply(find_stash_tags)

# Step 8: Add boolean columns for tags and performers
df_tags["has_local_tags_not_in_stashdb"] = df_tags["extra_tags"].apply(lambda x: len(x) > 0)
df_tags["has_stashdb_tags_not_in_local"] = df_tags["missing_tags"].apply(lambda x: len(x) > 0)

# Step 9: Select relevant columns for final dataframes
df_tags_selected = df_tags[[
    "scene_id", "existing_tags", "stashdb_tags",
    "missing_tags", "extra_tags", "stash_tags",
    "has_local_tags_not_in_stashdb",
    "has_stashdb_tags_not_in_local",
]]

In [ ]:
# Update tags for scenes where tags are not identical
for _index, row in df_selected.iterrows():
    if not row["tags_identical"] and row["stash_tags"]:
        stash.update_scene({
            "id": row["id"],
            "tag_ids": [tag["id"] for tag in row["stash_tags"] if tag]
        })